In [10]:
import autograd.numpy as np
from autograd.core import primitive
from autograd import grad, jacobian, hessian
from autograd.numpy.numpy_grads import unbroadcast
import scipy.stats

In [52]:
foo = [1, 2, 3, 4, 5]
print foo[0:2]
print foo[2:5]



x = np.matrix([1., 3., 4.]).T
x_info = np.outer(0.1 * x, x) + np.eye(3)
mu = np.matrix([0.1, 0.2, 0.5]).T
mu_cov = np.matrix(np.outer(mu, mu) + np.eye(3))
print mu
print mu_cov
print np.concatenate((mu.A1, mu_cov.A1))

[1, 2]
[3, 4, 5]
[[ 0.1]
 [ 0.2]
 [ 0.5]]
[[ 1.01  0.02  0.05]
 [ 0.02  1.04  0.1 ]
 [ 0.05  0.1   1.25]]
[ 0.1   0.2   0.5   1.01  0.02  0.05  0.02  1.04  0.1   0.05  0.1   1.25]


In [62]:
def LogLikelihood(x_row, x_info, e_mu, e_mu_cov):
    # TODO: if you're not using autodiff you can just do the matrix multiply once.
#     return 0.5 * (np.dot(e_mu.T, np.matmul(x_info, x_row)) + \
#                   np.dot(x_row.T, np.matmul(x_info, e_mu)) - \
#                   np.dot(e_mu.T, np.matmul(x_info, e_mu)) - \
#                   np.trace(np.matmul(x_info, e_mu_cov)))
    return e_mu[0] * x_row[0]

@primitive
def LogLikelihoodAD(x_row, x_info, e_mu, e_mu_cov):
    return LogLikelihood(x_row, x_info, e_mu, e_mu_cov)

def LogLikelihoodGrad_e_mu(x_row, x_info, e_mu):
    return np.matmul(x_info, x_row - e_mu)

def LogLikelihoodGrad_e_mu_cov(x_info):
    return x_info

def LogLikelihoodAD_e_mu_vjp(g, ans, vs, gvs, x_row, x_info, e_mu, e_mu_cov):
    return g * LogLikelihoodGrad_e_mu(x_row, x_info, e_mu)

def LogLikelihoodAD_e_mu_cov_vjp(g, ans, vs, gvs, x_row, x_info, e_mu, e_mu_cov):
    return g * LogLikelihoodGrad_e_mu_cov(x_info)

def ParamUnwrapper(vec, k):
    e_mu = vec[0:k]
    mu_cov = vec[k:(k + k ** 2)].reshape(k, k)
    return e_mu, mu_cov
    
def ParamWrapper(e_mu, mu_cov, k):
    return np.concatenate((e_mu.A1, mu_cov.A1))

def LogLikelihoodWrapper(vec, x_row, x_info):
    e_mu, mu_cov = ParamUnwrapper(vec, x_row.size)
    return LogLikelihood(x_row, x_info, e_mu, mu_cov)


LogLikelihoodAD.defvjp(LogLikelihoodAD_e_mu_vjp, argnum=2)
LogLikelihoodAD.defvjp(LogLikelihoodAD_e_mu_cov_vjp, argnum=3)

print LogLikelihood(x, x_info, mu, mu_cov)
print LogLikelihoodAD(x, x_info, mu, mu_cov)

LogLikelihoodGrad = grad(LogLikelihoodWrapper, argnum=0)
# LogLikelihoodGrad = grad(LogLikelihoodAD, argnum=2)
    
vec = ParamWrapper(mu, mu_cov, 3)
if False:
    mu_test, mu_cov_test = ParamUnwrapper(vec, 3)
    print vec
    print mu_test
    print mu_cov_test

print LogLikelihoodWrapper(vec, x_row, x_info)
LogLikelihoodGrad(vec, x, x_info)


[[ 0.1]]
[[ 0.1]]
0.1


array([ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.])

In [26]:

def MyFun(x):
    x_mat = np.matrix(x).T
    return 3 * np.dot(x_mat.T, x_mat)[0,0]

x = np.array([3., 2., 1.])
x_mat = np.matrix(x).T
print 3 * np.ravel(np.dot(x_mat.T, x_mat))
MyFunGrad = grad(MyFun)
MyFunGrad(x)

[ 42.]


array([ 18.,  12.,   6.])